# DenseNet Architecture

In [1]:
import os
import pathlib

import tensorflow as tf
from bcd.model.factory import DenseNetFactory
from bcd.model.visual import X4LearningVisualizer
from bcd.utils.model import thaw

ImportError: cannot import name 'learning_curve' from 'bcd.model.visual' (/home/john/projects/bcd/bcd/model/visual.py)

In [ ]:
batch_size = 32
input_shape = (224,224,3)
output_shape = 1

initial_epochs = 100  # Number of epochs to train for feature extraction

fine_tune_epochs = 10  # Number of epochs for each fine tune round
fine_tune_thaw_rate = 0.05  # Additional proportion of layers to unthaw each fine tune round. 

early_stop_monitor = "val_loss"  # Monitor validation loss for early stopping
early_stop_patience = 3  # The number of consecutive epochs for which lack of improvement is tolerated 
early_stop_restore_best_weights = True  # Returns the best weights rather than the weights at the last epoch.

learning_rate_base = 0.0001  # Base learning rate for the Adam optimizer 
learning_rate_decay = 10 # The factor by which learning rate decays for each fine tune round.

loss = "binary_crossentropy"
activation = "sigmoid"
metric = "accuracy"

train_dir_10 = pathlib.Path("data/image/1_final/training_10/training/").with_suffix('')
test_dir = pathlib.Path("data/image/1_final/test/test/").with_suffix('')
checkpoint_path = "models/checkpoints/densenet.ckpt"
model_directory = "models/densenet/"

## Preliminaries - Callbacks
Create a callback checkpoint that will automatically save model weights at each epoch.

In [ ]:
# Create a ModelCheckpoint callback that saves the model's weights only
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                         save_weights_only=True, # set to False to save the entire model
                                                         save_best_only=True, # save only the best model weights instead of a model every epoch
                                                         save_freq="epoch", # save every epoch
                                                         verbose=1)

In [ ]:
# Create an early stopping callback that will stop training if validation loss doesn't improve in n epochs
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor=early_stop_monitor, patience=early_stop_patience, restore_best_weights=early_stop_restore_best_weights)

## Load Data

In [ ]:
# Training DataSet (10%)
train_ds_10 = tf.keras.utils.image_dataset_from_directory(
    train_dir_10,
    labels="inferred",
    color_mode="rgb",
    image_size=(224,224),
    shuffle=True,
    validation_split=0.2,
    subset='training',
    interpolation="bilinear",
    seed=123,
    batch_size=batch_size)

# Validation DataSet (10%)
val_ds_10 = tf.keras.utils.image_dataset_from_directory(
    train_dir_10,
    labels="inferred",
    color_mode="rgb",
    image_size=(224,224),
    shuffle=True,
    validation_split=0.2,
    subset='validation',
    interpolation="bilinear",
    seed=123,
    batch_size=batch_size)

# Test Set
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels="inferred",
    color_mode="rgb",
    image_size=(224,224),
    shuffle=True)

## Feature Extraction

In [ ]:
factory = DenseNetFactory()
densenet = factory.create(input_shape=input_shape, output_shape=output_shape, learning_rate=learning_rate_base, trainable=False, loss=loss, activation=activation, metric=metric)
densenet_history = densenet.fit(train_ds_10, epochs=initial_epochs, validation_data=val_ds_10, callbacks=[checkpoint_callback, early_stop_callback])
filename = "densenet_feature_extraction.keras"
filepath = os.path.join(model_directory, filename)
densenet.save(filepath)

In [ ]:
x4v = X4LearningVisualizer(name="densenet")
x4v(history=densenet_history)

We have an training accuracy of 60% and validation accuracy of 70% on 10% of the data in just 5 epochs. 

In [ ]:
densenet.summary()

## Fine Tune

### Resume Strategy

#### Fine Tune Round 1

In [ ]:
fine_tune_round = 1
densenet_ft_epochs = initial_epochs + (fine_tune_epochs * fine_tune_round)

# Determine the number of layers to thaw.
densenet_base_layer = 5
densenet_base = densenet.layers[densenet_base_layer]
densenet_layers = densenet_base.layers
densenet_n_layers = len(densenet_layers)
thaw_n_layers = int(densenet_n_layers * fine_tune_thaw_rate) * fine_tune_round

# Freeze all but the top thaw_n_layers
densenet = thaw(n=thaw_n_layers, model=densenet, base_model_layer=densenet_base_layer)

# Recompile the model
densenet.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=(learning_rate_base/(learning_rate_decay**fine_tune_round))), metrics=[metric])

# Check trainable weights in summary
densenet.summary()


In [ ]:

densenet_history_ft_1 = densenet.fit(train_ds_10, epochs=densenet_ft_epochs, validation_data=val_ds_10, 
                                         initial_epoch=densenet_history.epoch[-1],
                                         callbacks=[checkpoint_callback, early_stop_callback])
filename = "densenet_fine_tuning_1_1.keras"
filepath = os.path.join(model_directory, filename)
densenet.save(filepath)

#### Fine Tune Round 2

In [ ]:
fine_tune_round = 2
densenet_ft_epochs = initial_epochs + (fine_tune_epochs * fine_tune_round)

# Determine the number of layers to thaw.
thaw_n_layers = thaw_n_layers * fine_tune_round

# Freeze all but the top thaw_n_layers
densenet = thaw(n=thaw_n_layers, model=densenet, base_model_layer=densenet_base_layer)

# Recompile the model
densenet.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=(learning_rate_base/(learning_rate_decay**fine_tune_round))), metrics=[metric])

# Check trainable weights in summary
densenet.summary()

In [ ]:

densenet_history_ft_2 = densenet.fit(train_ds_10, epochs=densenet_ft_epochs, validation_data=val_ds_10, 
                                         initial_epoch=densenet_history_ft_1.epoch[-1],
                                         callbacks=[checkpoint_callback, early_stop_callback])
filename = "densenet_fine_tuning_1_2.keras"
filepath = os.path.join(model_directory, filename)
densenet.save(filepath)

#### Fine Tune Round 3

In [ ]:
fine_tune_round = 3
densenet_ft_epochs = initial_epochs + (fine_tune_epochs * fine_tune_round)

# Determine the number of layers to thaw.
thaw_n_layers = thaw_n_layers * fine_tune_round

# Freeze all but the top thaw_n_layers
densenet = thaw(n=thaw_n_layers, model=densenet, base_model_layer=densenet_base_layer)

# Recompile the model
densenet.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=(learning_rate_base/(learning_rate_decay**fine_tune_round))), metrics=[metric])

# Check trainable weights in summary
densenet.summary()

In [ ]:

densenet_history_ft_3 = densenet.fit(train_ds_10, epochs=densenet_ft_epochs, validation_data=val_ds_10, 
                                         initial_epoch=densenet_history_ft_2.epoch[-1],
                                         callbacks=[checkpoint_callback, early_stop_callback])
filename = "densenet_fine_tuning_1_3.keras"
filepath = os.path.join(model_directory, filename)
densenet.save(filepath)

### Restart Strategy

#### Fine Tuning Round 1

In [ ]:
# Reload Weights from Feature Extraction Stage
filename = "densenet_feature_extraction.keras"
filepath = os.path.join(model_directory, filename)
densenet = tf.keras.models.load_model(filepath)


In [ ]:
fine_tune_round = 1
densenet_ft_epochs = initial_epochs + (fine_tune_epochs * fine_tune_round)

# Determine the number of layers to thaw.
thaw_n_layers = thaw_n_layers * fine_tune_round

# Freeze all but the top thaw_n_layers
densenet = thaw(n=thaw_n_layers, model=densenet, base_model_layer=densenet_base_layer)

# Recompile the model
densenet.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=(learning_rate_base/(learning_rate_decay**fine_tune_round))), metrics=[metric])

# Check trainable weights in summary
densenet.summary()

In [ ]:

densenet_history_ft_2_1 = densenet.fit(train_ds_10, epochs=densenet_ft_epochs, validation_data=val_ds_10, 
                                         initial_epoch=densenet_history.epoch[-1],
                                         callbacks=[checkpoint_callback, early_stop_callback])
filename = "densenet_fine_tuning_2_1.keras"
filepath = os.path.join(model_directory, filename)
densenet.save(filepath)

#### Fine Tuning Round 2

In [ ]:
fine_tune_round = 2
densenet_ft_epochs = initial_epochs + (fine_tune_epochs * fine_tune_round)

# Determine the number of layers to thaw.
thaw_n_layers = thaw_n_layers * fine_tune_round

# Freeze all but the top thaw_n_layers
densenet = thaw(n=thaw_n_layers, model=densenet, base_model_layer=densenet_base_layer)

# Recompile the model
densenet.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=(learning_rate_base/(learning_rate_decay**fine_tune_round))), metrics=[metric])

# Check trainable weights in summary
densenet.summary()

In [ ]:

densenet_history_ft_2_2 = densenet.fit(train_ds_10, epochs=densenet_ft_epochs, validation_data=val_ds_10, 
                                         initial_epoch=densenet_history_ft_2_1.epoch[-1],
                                         callbacks=[checkpoint_callback, early_stop_callback])
filename = "densenet_fine_tuning_2_2.keras"
filepath = os.path.join(model_directory, filename)
densenet.save(filepath)

#### Fine Tuning Round 3

In [ ]:
fine_tune_round = 3
densenet_ft_epochs = initial_epochs + (fine_tune_epochs * fine_tune_round)

# Determine the number of layers to thaw.
thaw_n_layers = thaw_n_layers * fine_tune_round

# Freeze all but the top thaw_n_layers
densenet = thaw(n=thaw_n_layers, model=densenet, base_model_layer=densenet_base_layer)

# Recompile the model
densenet.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=(learning_rate_base/(learning_rate_decay**fine_tune_round))), metrics=[metric])

# Check trainable weights in summary
densenet.summary()

In [ ]:

densenet_history_ft_2_3 = densenet.fit(train_ds_10, epochs=densenet_ft_epochs, validation_data=val_ds_10, 
                                         initial_epoch=densenet_history_ft_2_2.epoch[-1],
                                         callbacks=[checkpoint_callback, early_stop_callback])
filename = "densenet_fine_tuning_2_3.keras"
filepath = os.path.join(model_directory, filename)
densenet.save(filepath)

In [ ]:
densenet_history_ft_2_3.history